# Building a Language Model Application: Chat Models

Similarly, you can use chat models instead of LLMs. Chat models are a variation on language models.

## Get Message completions from a Chat Model

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

chat = ChatOpenAI(temperature=0)
chat(
    [
        HumanMessage(
            content="Translate this sentence to German: I don" "t understand a thing"
        )
    ]
)

AIMessage(content='Ich verstehe nichts.', additional_kwargs={})

We can pass in multiple messages for OpenAI's `gpt-3.5-turbo` and `gpt-4` models. The default for `ChatOpenAI` model is `gpt-3.5-turbo`.

In [2]:
messages = [
    SystemMessage(
        content="You are a helpful assistant that translates English to German"
    ),
    HumanMessage(
        content="Translate this sentence from English to German. I love programming"
    ),
]

chat(messages)

AIMessage(content='Ich liebe Programmieren.', additional_kwargs={})

We can also generate completions for multiple sets of messages using `generate`.

In [5]:
batch_messages = [
    [
        SystemMessage(
            content="You are a helpful assistant that translates English to German"
        ),
        HumanMessage(
            content="Translate this sentence from English to German. I love programming"
        ),
    ],
    [
        SystemMessage(
            content="You are a helpful assistant that translates English to Malay"
        ),
        HumanMessage(
            content="Translate this sentence from English to Malay. I love programming"
        ),
    ],
]

result = chat.generate(batch_messages)
result

LLMResult(generations=[[ChatGeneration(text='Ich liebe programmieren.', generation_info=None, message=AIMessage(content='Ich liebe programmieren.', additional_kwargs={}))], [ChatGeneration(text='Saya suka pemrograman.', generation_info=None, message=AIMessage(content='Saya suka pemrograman.', additional_kwargs={}))]], llm_output={'token_usage': {'prompt_tokens': 68, 'completion_tokens': 13, 'total_tokens': 81}, 'model_name': 'gpt-3.5-turbo'})

We can recover things like token usage from the `LLMResult`:

In [6]:
result.llm_output["token_usage"]

{'prompt_tokens': 68, 'completion_tokens': 13, 'total_tokens': 81}

## Chat Prompt Templates


We can make use of templating using `MessagePromptTemplate`.

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

chat = ChatOpenAI(temperature=0)

template = (
    "You are a helpful assistant that translates {input_language} to {output_language}"
)
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

# Get a chat completion from the formatted message.
chat(
    chat_prompt.format_prompt(
        input_language="English", output_language="German", text="I have a dream"
    ).to_messages()
)

AIMessage(content='Ich habe einen Traum.', additional_kwargs={})

## Chains with Chat Models

We can also use `LLMChain` with the chat model.

In [11]:
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

chat = ChatOpenAI(temperature=0)

template = (
    "You are a helpful assistant that translates {input_language} to {output_language}"
)
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

chain = LLMChain(llm=chat, prompt=chat_prompt)
chain.run(input_language="English", output_language="Malay", text="I love programming")

'Saya suka memprogram.'

## Agents with Chat Models

We can also use chat models with Agents.

In [16]:
from langchain.agents import initialize_agent, load_tools
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

# First, we load the language model we are going to use to control the agent.
chat = ChatOpenAI(temperature=0)

# Next, we load some tools to use. Note that `llm-math` tool uses an LLM, so we need to pass that in.
llm = OpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Finally, we initialize an agent with the tools, the language model and the type of agent we want to use.
agent = initialize_agent(tools, chat, agent="zero-shot-react-description", verbose=True)

agent.run(
    "What is the latest movies screening in Singapore, and when was it? How many days was it since the last movie, multiply with 2.5?"
)



> Entering new AgentExecutor chain...
I need to find information about the latest movie screening in Singapore and perform a calculation based on the number of days since the last movie.
Action 1: Search
Action Input 1: "Latest movie screening Singapore"
Observation 1: The latest movie screening in Singapore is "Tenet" and it premiered on August 27, 2020.
Thought: I need to calculate the number of days since the last movie and multiply it by 2.5
Action 2: Calculator
Action Input 2: (Current date - last movie screening date) x 2.5
Observation 2: (Current date - August 19, 2020) x 2.5 = 28.75
Thought: I now know the final answer
Final Answer: It has been 28.75 days since the last movie screening, multiplied by 2.5 equals 71.9.

> Finished chain.


'It has been 28.75 days since the last movie screening, multiplied by 2.5 equals 71.9.'

## Memory: Add state to Chains and Agents

We can use Memory with chains and agents initialized with chat models. The main difference between this and Memory for LLMs is that rather than trying to condense all previous messages into a string, we can keep them as their own unique memory object.

In [19]:
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            "The following is a friendly conversation between a human and an AI. The AI is talktative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know."
        ),
        MessagesPlaceholder(variable_name="history"),
        HumanMessagePromptTemplate.from_template("{input}"),
    ]
)
llm = ChatOpenAI(temperature=0)
memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(memory=memory, prompt=prompt, llm=llm)

conversation.predict(input="Hi there!")

'Hello! How can I assist you today?'

In [20]:
conversation.predict(input="Tell me more about yourself")

'Sure! I am an AI language model designed to process natural language input and provide relevant responses. I was created using advanced machine learning techniques that enable me to learn from vast amounts of data, and continuously improve my abilities over time. My primary function is to help people with a wide range of tasks, from answering questions to providing customer support and even offering suggestions on what to eat for dinner. Is there anything specific you would like to know about me?'